In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
from pprint import pprint
from config import g_key

In [4]:
cameras_file = os.path.join("../Desktop/Archive", "red-light-camera-locations.csv")
violations_file = os.path.join("../Desktop/Archive","red-light-camera-violations.csv")

In [10]:
red_locs_df = pd.read_csv(cameras_file)
violations_file = pd.read_csv(speedings_file)

In [11]:
violations_file.columns

Index(['INTERSECTION', 'CAMERA ID', 'ADDRESS', 'VIOLATION DATE', 'VIOLATIONS',
       'X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'Historical Wards 2003-2015', 'Zip Codes', 'Community Areas',
       'Census Tracts', 'Wards'],
      dtype='object')

In [12]:
cams_per_ward = pd.DataFrame(red_locs_df.groupby("Wards")["Intersection"].count())
violations_per_ward = pd.DataFrame(violations_file.groupby("Wards")["VIOLATIONS"].sum())
wards = [
1,
2,
3,
4,
5,
6,
7,
8,
9,
10,
11,
12,
13,
14,
15,
16,
17,
18,
19,
20,
21,
22,
23,
24,
25,
26,
27,
28,
29,
30,
31,
32,
33,
34,
35,
36, 
37,  
38, 
40,
41,
44,
45,
46,
48,
49,
50]

In [14]:
merge_df = pd.merge(cams_per_ward, violations_per_ward, on="Wards")
merge_df = merge_df.rename(columns={
    "Intersection": "Number of Red Light Cams per Ward",
    "VIOLATIONS": "Number of Red Light Violations per Ward"
})
merge_df.head()

,Number of Red Light Cams per Ward,Number of Red Light Violations per Ward
Wards,,
1,2,33960
2,2,25681
3,2,34327
4,1,30204
5,1,21369


In [16]:
merge_df["Ward Number"] = wards
merge_df.head()

,Number of Red Light Cams per Ward,Number of Red Light Violations per Ward,Ward Number
Wards,,,
1,2,33960,1
2,2,25681,2
3,2,34327,3
4,1,30204,4
5,1,21369,5


In [18]:
ward_locs = os.path.join("../Desktop/Archive", "ward_offices_locs_alt.csv")
ward_locs_df = pd.read_csv(ward_locs)
ward_locs_df = ward_locs_df.rename(columns={
    "Ward":"Ward Number"
})
ward_locs_df.head()

,Ward Number,Latitude,Longitude,Neighborhoods,Navigator
0,1,41.918127,-87.687821,"Wicker Park, West Town, Ukranian Village, Loga...","N Side, W and Near W Side"
1,2,41.908759,-87.668029,"Near North Side, Lincoln Park, Wicker Park, Uk...","N Side, W and Near W Side"
2,3,41.804552,-87.626387,"Hyde Park, Bronzeville, Fuller Park and Grand...",Near S Side
3,4,41.885555,-87.631870,"South Loop, Museum Campus, Douglas, Grand Boul...",S Side
4,5,41.766729,-87.568063,"Indian Village, Hyde Park, Jackson Park, South...",S Side


In [20]:
new_merge_df = pd.merge(merge_df, ward_locs_df, on="Ward Number")
new_merge_df.head()

,Number of Red Light Cams per Ward,Number of Red Light Violations per Ward,Ward Number,Latitude,Longitude,Neighborhoods,Navigator
0,2,33960,1,41.918127,-87.687821,"Wicker Park, West Town, Ukranian Village, Loga...","N Side, W and Near W Side"
1,2,25681,2,41.908759,-87.668029,"Near North Side, Lincoln Park, Wicker Park, Uk...","N Side, W and Near W Side"
2,2,34327,3,41.804552,-87.626387,"Hyde Park, Bronzeville, Fuller Park and Grand...",Near S Side
3,1,30204,4,41.885555,-87.631870,"South Loop, Museum Campus, Douglas, Grand Boul...",S Side
4,1,21369,5,41.766729,-87.568063,"Indian Village, Hyde Park, Jackson Park, South...",S Side


In [22]:
locations = new_merge_df[["Latitude", "Longitude"]]
locations.head()

,Latitude,Longitude
0,41.918127,-87.687821
1,41.908759,-87.668029
2,41.804552,-87.626387
3,41.885555,-87.631870
4,41.766729,-87.568063


In [25]:
gmaps.configure(api_key = g_key)
speed_cams = new_merge_df["Number of Red Light Cams per Ward"].astype(float)

In [27]:
new_merge_df.head()

,Number of Red Light Cams per Ward,Number of Red Light Violations per Ward,Ward Number,Latitude,Longitude,Neighborhoods,Navigator
0,2,33960,1,41.918127,-87.687821,"Wicker Park, West Town, Ukranian Village, Loga...","N Side, W and Near W Side"
1,2,25681,2,41.908759,-87.668029,"Near North Side, Lincoln Park, Wicker Park, Uk...","N Side, W and Near W Side"
2,2,34327,3,41.804552,-87.626387,"Hyde Park, Bronzeville, Fuller Park and Grand...",Near S Side
3,1,30204,4,41.885555,-87.631870,"South Loop, Museum Campus, Douglas, Grand Boul...",S Side
4,1,21369,5,41.766729,-87.568063,"Indian Village, Hyde Park, Jackson Park, South...",S Side


In [29]:
info_box_template = """
<dl>
<dt>Ward</dt><dd>{Ward Number}</dd>
<dt>Ward Location<dd>{Navigator}</dd>
<dt>Neighborhoods<dd>{Neighborhoods}</dd>
<dt>Number of Red Light Cams</dt><dd>{Number of Red Light Cams per Ward}</dd>
<dt>Number of Red Light Violations per Ward</dt><dd>{Number of Red Light Violations per Ward}</dd>
</dl>
"""

In [31]:
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights= speed_cams, dissipating=False, max_intensity=1, point_radius=0.01)
fig.add_layer(heat_map)
cam_info = [info_box_template.format(**row) for index, row in new_merge_df.iterrows()]
markers = gmaps.marker_layer(locations, info_box_content= cam_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
new_merge_df.sort_values(by="Number of Red Light Violations per Ward", ascending=False).reset_index(drop=True)

,Number of Red Light Cams per Ward,Number of Red Light Violations per Ward,Ward Number,Latitude,Longitude,Neighborhoods,Navigator
0,5,173116,28,41.879433,-87.692469,"West Garfield Park, East Garfield Park, Austin...",W and Near W Side
1,5,163190,13,41.776442,-87.722318,"West Lawn, Clearing, West Elsdon, Garfield Ridge",SW Side
2,6,155493,23,41.794699,-87.775670,"West Elsdon, West Lawn, Garfield Ridge and Cle...",Far W Side and SW Side
3,3,116923,33,41.954898,-87.703292,"Albany Park, Irving Park and Avondale","N Side, NW Side"
4,2,108233,25,41.859393,-87.663994,"Pilsen, Greek Town, Chinatown, University Vill...",Lower W Side
5,2,106413,32,41.931614,-87.675177,"Wicker Park, Logan Square, Bucktown, Roscoe Vi...",N Side
6,7,106272,40,41.985173,-87.689067,"Lincoln Square, Edgewater, West Ridge",Far N. Side
7,6,100330,46,41.965654,-87.656924,"Buena Park, Lakeview, Uptown","Far N. Side, N. Side"
8,3,92851,22,41.849848,-87.712418,"South Lawndalwe, Little Village, Archer Heights",W and Near W Side
9,3,92006,20,41.791594,-87.630251,"Back of the Yards, Canaryville, Washington Par...",S and SW Side
